Example notebook

In [27]:
import os
from pathlib import Path, PosixPath

project_name = 'myproj'
parent_dir = 'tmp2'
lookup_dir = Path(parent_dir) / project_name

venv_dir = str(Path(lookup_dir) / '.venv')
file_list = set()
for root, _dirs, files in os.walk(lookup_dir):
	_dirs = sorted(_dirs)
	print(f'* Root: {root}')
	print(f'Dirs: {_dirs}')
	print(f'Files: {files}')
	if not root.startswith(venv_dir):
		for file in files:
			print(f'  {file}')
			file_list.add(Path(root) / file)

file_list

* Root: tmp2/myproj
Dirs: ['.venv', 'conf', 'data', 'myproj', 'notebooks', 'stupid dir']
Files: ['README.md', '.gitignore', 'pyproject.toml']
  README.md
  .gitignore
  pyproject.toml
* Root: tmp2/myproj/data
Dirs: []
Files: ['example_data.csv']
  example_data.csv
* Root: tmp2/myproj/myproj
Dirs: []
Files: ['example.py']
  example.py
* Root: tmp2/myproj/stupid dir
Dirs: []
Files: []
* Root: tmp2/myproj/conf
Dirs: []
Files: ['workflow.yaml', 'config.yaml']
  workflow.yaml
  config.yaml
* Root: tmp2/myproj/.venv
Dirs: ['bin', 'lib']
Files: ['pyvenv.cfg', '.gitignore']
* Root: tmp2/myproj/.venv/lib
Dirs: ['python3.11']
Files: []
* Root: tmp2/myproj/.venv/lib/python3.11
Dirs: ['site-packages']
Files: []
* Root: tmp2/myproj/.venv/lib/python3.11/site-packages
Dirs: ['__pycache__', '_distutils_hack', 'myproj-0.1.0.dist-info', 'pip', 'pip-23.2.1.dist-info', 'pkg_resources', 'setuptools', 'setuptools-68.0.0.dist-info', 'wheel', 'wheel-0.41.0.dist-info']
Files: ['wheel-0.41.0.virtualenv', 'pip-2

{PosixPath('tmp2/myproj/.gitignore'),
 PosixPath('tmp2/myproj/README.md'),
 PosixPath('tmp2/myproj/conf/config.yaml'),
 PosixPath('tmp2/myproj/conf/workflow.yaml'),
 PosixPath('tmp2/myproj/data/example_data.csv'),
 PosixPath('tmp2/myproj/myproj/example.py'),
 PosixPath('tmp2/myproj/notebooks/example_notebook.ipynb'),
 PosixPath('tmp2/myproj/pyproject.toml')}

In [38]:
def dirs_and_files_from_path(
	path: PosixPath,
) -> tuple[list[PosixPath], list[PosixPath]]:

	dirs = []
	files = []
	for file_or_dir in path.iterdir():
		if file_or_dir.is_file():
			files.append(file_or_dir) 
		if file_or_dir.is_dir():
			dirs.append(file_or_dir)

	return (sorted(dirs), sorted(files))

print(dirs_and_files_from_path(Path('tmp2/myproj')))

([PosixPath('tmp2/myproj/.venv'), PosixPath('tmp2/myproj/conf'), PosixPath('tmp2/myproj/data'), PosixPath('tmp2/myproj/myproj'), PosixPath('tmp2/myproj/notebooks'), PosixPath('tmp2/myproj/stupid dir')], [PosixPath('tmp2/myproj/.gitignore'), PosixPath('tmp2/myproj/README.md'), PosixPath('tmp2/myproj/pyproject.toml')])


In [15]:
import re

from directory_tree import display_tree
from bluprint.colors import Style


In [20]:
dir_re = re.compile(r'(\s[^/\n]+)/')

def style_directory(string: str) -> str:
    return f'{Style.blue}{Style.bold}{string}{Style.end}'

project_tree = display_tree(
	'tmp2/myproj/',
	show_hidden=True,
	ignore_list=['.venv'],
	string_rep=True, 
)

project_tree = dir_re.sub(
    f'{Style.blue}{Style.bold}\\1{Style.end}',
    project_tree,
)

print(project_tree)


myproj/
├── .gitignore
├── conf
│   ├── config.yaml
│   └── workflow.yaml
├── data
│   └── example_data.csv
├── myproj
│   └── example.py
├── notebooks
│   └── example_notebook.ipynb
├── pyproject.toml
├── README.md
└── stupid dir



In [48]:
from importlib.readers import MultiplexedPath

In [50]:
MultiplexedPath(Path.home())

MultiplexedPath('/home/igor')

In [47]:
from importlib import resources

resources.files('demo').parent.parent

AttributeError: 'MultiplexedPath' object has no attribute 'parent'